In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgb
import optuna.integration.lightgbm as olgb

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
train_pre = pd.read_csv('../data/train_pre.csv')
test_pre = pd.read_csv('../data/test_pre.csv')

train = pd.read_csv('../data/train_data.csv')
test = pd.read_csv('../data/test_data.csv')

In [4]:
train_pre.drop(columns='Unnamed: 0', inplace=True)
test_pre.drop(columns='Unnamed: 0', inplace=True)

train_pre.head()

,position,age,area,sex,partner,num_child,education,service_length,study_time,commute,...,sex_and_position,education_and_position,age_diff_service_length,age_layer,agelayer_and_position,overtime_by_service_length,overtime_by_age,study_time_by_service_length,study_time_by_age,age_by_service_length
0,1,44,愛知県,2,1,2,1,24,2.0,0.6,...,2_1,1_1,1.760000,2,2_1,0.368000,0.209091,0.080000,0.045455,1.760000
1,2,31,奈良県,1,0,0,0,13,9.0,0.7,...,1_2,0_2,2.214286,2,2_2,0.885714,0.400000,0.642857,0.290323,2.214286
2,2,36,山口県,1,0,0,2,14,4.0,0.4,...,1_2,2_2,2.400000,2,2_2,1.126667,0.469444,0.266667,0.111111,2.400000
3,0,22,東京都,2,0,0,0,4,3.0,0.0,...,2_0,0_0,4.400000,1,1_0,1.220000,0.277273,0.600000,0.136364,4.400000
4,0,25,鹿児島県,2,0,0,1,5,3.0,0.2,...,2_0,1_0,4.166667,1,1_0,0.816667,0.196000,0.500000,0.120000,4.166667


In [5]:
target = train['salary']

In [6]:
from sklearn.preprocessing import LabelEncoder
cat_features = ['area', 'sex_and_position', 'education_and_position', 'agelayer_and_position']

le = LabelEncoder()
for col in cat_features:
    train_pre[col] = le.fit_transform(train_pre[col])
    test_pre[col] = le.fit_transform(test_pre[col])

In [7]:
train_pre.head()

,position,age,area,sex,partner,num_child,education,service_length,study_time,commute,...,sex_and_position,education_and_position,age_diff_service_length,age_layer,agelayer_and_position,overtime_by_service_length,overtime_by_age,study_time_by_service_length,study_time_by_age,age_by_service_length
0,1,44,24,2,1,2,1,24,2.0,0.6,...,6,6,1.760000,2,8,0.368000,0.209091,0.080000,0.045455,1.760000
1,2,31,10,1,0,0,0,13,9.0,0.7,...,2,2,2.214286,2,9,0.885714,0.400000,0.642857,0.290323,2.214286
2,2,36,14,1,0,0,2,14,4.0,0.4,...,2,12,2.400000,2,9,1.126667,0.469444,0.266667,0.111111,2.400000
3,0,22,26,2,0,0,0,4,3.0,0.0,...,5,0,4.400000,1,3,1.220000,0.277273,0.600000,0.136364,4.400000
4,0,25,46,2,0,0,1,5,3.0,0.2,...,5,5,4.166667,1,3,0.816667,0.196000,0.500000,0.120000,4.166667


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_pre, target, test_size=0.3)

x_train.shape
x_test.shape
y_train.shape
y_test.shape

(14700, 22)

(6300, 22)

(14700,)

(6300,)

In [9]:

trains = lgb.Dataset(x_train, y_train)
valids = lgb.Dataset(x_test, y_test, reference=trains)


trains = olgb.Dataset(x_train, y_train)
valids = olgb.Dataset(x_test, y_test, reference=trains)

In [58]:
# LightGBMのパラメータ設定
params = {'objective': 'regression',
          'metric': 'l1',
          'verbosity': -1,
          'boosting_type': 'gbdt',
          'feature_pre_filter': False,
          'lambda_l1': 3.92994872964793e-05,
          'lambda_l2': 0.0008522013002932,
          'num_leaves': 68,
          'feature_fraction': 0.88,
          'bagging_fraction': 1.0,
          'bagging_freq': 0,
          'min_child_samples': 5,
          'num_iterations': 1000,
          'early_stopping_round': 100}


# LightGBM学習
gbm = lgb.train(params,
                trains,
                num_boost_round=200,
                valid_sets=valids,
                early_stopping_rounds=100
               )

[1]	valid_0's l1: 124.193
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 112.753
[3]	valid_0's l1: 102.421
[4]	valid_0's l1: 93.215
[5]	valid_0's l1: 85.1172
[6]	valid_0's l1: 77.8528
[7]	valid_0's l1: 71.4206
[8]	valid_0's l1: 65.5938
[9]	valid_0's l1: 61.335
[10]	valid_0's l1: 56.7178
[11]	valid_0's l1: 52.6602
[12]	valid_0's l1: 49.0102
[13]	valid_0's l1: 45.7127
[14]	valid_0's l1: 42.8677
[15]	valid_0's l1: 40.3647
[16]	valid_0's l1: 38.1735
[17]	valid_0's l1: 36.6899
[18]	valid_0's l1: 34.8938
[19]	valid_0's l1: 33.2793
[20]	valid_0's l1: 32.2898
[21]	valid_0's l1: 31.4071
[22]	valid_0's l1: 30.2439
[23]	valid_0's l1: 29.2232
[24]	valid_0's l1: 28.6943
[25]	valid_0's l1: 27.9834
[26]	valid_0's l1: 27.3609
[27]	valid_0's l1: 26.8333
[28]	valid_0's l1: 26.3235
[29]	valid_0's l1: 25.8737
[30]	valid_0's l1: 25.5345
[31]	valid_0's l1: 25.1974
[32]	valid_0's l1: 24.9556
[33]	valid_0's l1: 24.74
[34]	valid_0's l1: 24.5192
[35]	valid_0's l1: 24.3793
[36]	v

In [60]:
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

In [62]:
y_pred

gbm_scores = calculate_scores(y_test, y_pred)
print(gbm_scores)

array([349.98515996, 274.54358595, 475.47189709, ..., 558.89891787,
       171.24837743, 213.92791015])

              R2        MAE          MSE       RMSE
scores  0.961197  22.512628  1139.313376  33.753716


In [15]:
# optunaチューニング
params = {
        'objective': 'regression',
        'metric': 'mae',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }


best_params, history = {}, []
model = olgb.train(params,
                   trains,
                   valid_sets=valids
                  )

[I 2021-04-22 15:27:00,424] A new study created in memory with name: no-name-ca75a0ce-9d2f-4900-a2a0-ae8703cacbfb

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l1: 127.776
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 118.651
[3]	valid_0's l1: 110.994
[4]	valid_0's l1: 104.261
[5]	valid_0's l1: 97.9387
[6]	valid_0's l1: 92.4118
[7]	valid_0's l1: 85.0588
[8]	valid_0's l1: 78.113
[9]	valid_0's l1: 74.5205
[10]	valid_0's l1: 69.0573
[11]	valid_0's l1: 64.5304
[12]	valid_0's l1: 60.232
[13]	valid_0's l1: 58.1914
[14]	valid_0's l1: 56.2829
[15]	valid_0's l1: 53.05
[16]	valid_0's l1: 51.3844
[17]	valid_0's l1: 50.0002
[18]	valid_0's l1: 48.8761
[19]	valid_0's l1: 47.9336
[20]	valid_0's l1: 47.0242
[21]	valid_0's l1: 46.3951
[22]	valid_0's l1: 44.153
[23]	valid_0's l1: 42.2163
[24]	valid_0's l1: 41.5869
[25]	valid_0's l1: 40.0003
[26]	valid_0's l1: 38.4394
[27]	valid_0's l1: 37.1885
[28]	valid_0's l1: 36.064
[29]	valid_0's l1: 35.7241
[30]	valid_0's l1: 34.8285
[31]	valid_0's l1: 33.8277
[32]	valid_0's l1: 33.0582
[33]	valid_0's l1: 32.4597
[34]	valid_0's l1: 31.8321
[35]	valid_0's l1: 31.5795
[36]	val


feature_fraction, val_score: 23.542409:  14%|#4        | 1/7 [00:00<00:03,  1.70it/s][I 2021-04-22 15:27:01,016] Trial 0 finished with value: 23.542408902065908 and parameters: {'feature_fraction': 0.4}. Best is trial 0 with value: 23.542408902065908.

feature_fraction, val_score: 23.542409:  14%|#4        | 1/7 [00:00<00:03,  1.70it/s]

[1]	valid_0's l1: 124.771
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 115.423
[3]	valid_0's l1: 105.144
[4]	valid_0's l1: 98.0517
[5]	valid_0's l1: 91.66
[6]	valid_0's l1: 84.0209
[7]	valid_0's l1: 77.173
[8]	valid_0's l1: 71.0456
[9]	valid_0's l1: 67.2608
[10]	valid_0's l1: 62.2879
[11]	valid_0's l1: 58.3124
[12]	valid_0's l1: 54.499
[13]	valid_0's l1: 51.0484
[14]	valid_0's l1: 48.8974
[15]	valid_0's l1: 46.1557
[16]	valid_0's l1: 44.4521
[17]	valid_0's l1: 43.0505
[18]	valid_0's l1: 41.8629
[19]	valid_0's l1: 40.8189
[20]	valid_0's l1: 39.8721
[21]	valid_0's l1: 39.0547
[22]	valid_0's l1: 37.572
[23]	valid_0's l1: 36.2336
[24]	valid_0's l1: 35.6499
[25]	valid_0's l1: 34.5788
[26]	valid_0's l1: 33.3919
[27]	valid_0's l1: 32.6503
[28]	valid_0's l1: 31.8536
[29]	valid_0's l1: 31.5317
[30]	valid_0's l1: 30.8965
[31]	valid_0's l1: 30.2151
[32]	valid_0's l1: 29.726
[33]	valid_0's l1: 29.2705
[34]	valid_0's l1: 28.8225
[35]	valid_0's l1: 28.6591
[36]	val


feature_fraction, val_score: 23.020479:  29%|##8       | 2/7 [00:00<00:02,  1.91it/s][I 2021-04-22 15:27:01,392] Trial 1 finished with value: 23.020479283797442 and parameters: {'feature_fraction': 0.5}. Best is trial 1 with value: 23.020479283797442.

feature_fraction, val_score: 23.020479:  29%|##8       | 2/7 [00:00<00:02,  1.91it/s]

...


[195]	valid_0's l1: 23.5816
[196]	valid_0's l1: 23.5824
[197]	valid_0's l1: 23.5877
[198]	valid_0's l1: 23.5944
[199]	valid_0's l1: 23.5946
[200]	valid_0's l1: 23.5922
[201]	valid_0's l1: 23.5961
Early stopping, best iteration is:
[101]	valid_0's l1: 23.3878
[1]	valid_0's l1: 124.22
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 112.789
[3]	valid_0's l1: 102.519
[4]	valid_0's l1: 93.3337
[5]	valid_0's l1: 85.1592
[6]	valid_0's l1: 77.9063
[7]	valid_0's l1: 71.4967
[8]	valid_0's l1: 65.6251
[9]	valid_0's l1: 61.3582
[10]	valid_0's l1: 56.7668
[11]	valid_0's l1: 52.7225
[12]	valid_0's l1: 49.0776
[13]	valid_0's l1: 45.8154
[14]	valid_0's l1: 43.0283
[15]	valid_0's l1: 40.5294
[16]	valid_0's l1: 38.3203
[17]	valid_0's l1: 36.7944
[18]	valid_0's l1: 34.9377
[19]	valid_0's l1: 33.3785
[20]	valid_0's l1: 32.377
[21]	valid_0's l1: 31.4977
[22]	valid_0's l1: 30.3462
[23]	valid_0's l1: 29.3686
[24]	valid_0's l1: 28.8176
[25]	valid_0's l1: 28.1078
[26]	valid_0's


min_data_in_leaf, val_score: 22.512628:  80%|########  | 4/5 [00:02<00:00,  1.81it/s][I 2021-04-22 15:27:30,138] Trial 66 finished with value: 22.531363610151235 and parameters: {'min_child_samples': 10}. Best is trial 64 with value: 22.512628043393622.

min_data_in_leaf, val_score: 22.512628:  80%|########  | 4/5 [00:02<00:00,  1.81it/s]

[1]	valid_0's l1: 124.3
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 112.86
[3]	valid_0's l1: 102.641
[4]	valid_0's l1: 93.4828
[5]	valid_0's l1: 85.3039
[6]	valid_0's l1: 78.0146
[7]	valid_0's l1: 71.6115
[8]	valid_0's l1: 65.9051
[9]	valid_0's l1: 61.666
[10]	valid_0's l1: 57.0031
[11]	valid_0's l1: 52.9668
[12]	valid_0's l1: 49.254
[13]	valid_0's l1: 45.9663
[14]	valid_0's l1: 43.1843
[15]	valid_0's l1: 40.659
[16]	valid_0's l1: 38.4634
[17]	valid_0's l1: 36.9822
[18]	valid_0's l1: 35.1605
[19]	valid_0's l1: 33.581
[20]	valid_0's l1: 32.5994
[21]	valid_0's l1: 31.7013
[22]	valid_0's l1: 30.5262
[23]	valid_0's l1: 29.5526
[24]	valid_0's l1: 29.011
[25]	valid_0's l1: 28.3152
[26]	valid_0's l1: 27.6703
[27]	valid_0's l1: 27.1678
[28]	valid_0's l1: 26.6476
[29]	valid_0's l1: 26.2269
[30]	valid_0's l1: 25.8403
[31]	valid_0's l1: 25.5008
[32]	valid_0's l1: 25.2317
[33]	valid_0's l1: 25.0561
[34]	valid_0's l1: 24.8315
[35]	valid_0's l1: 24.6957
[36]	valid


min_data_in_leaf, val_score: 22.512628: 100%|##########| 5/5 [00:02<00:00,  2.03it/s]


[108]	valid_0's l1: 22.6791
[109]	valid_0's l1: 22.6731
[110]	valid_0's l1: 22.6805
[111]	valid_0's l1: 22.6773
[112]	valid_0's l1: 22.6725
[113]	valid_0's l1: 22.6831
[114]	valid_0's l1: 22.69
[115]	valid_0's l1: 22.6913
[116]	valid_0's l1: 22.6872
[117]	valid_0's l1: 22.6826
[118]	valid_0's l1: 22.6822
[119]	valid_0's l1: 22.6883
[120]	valid_0's l1: 22.6835
[121]	valid_0's l1: 22.6842
[122]	valid_0's l1: 22.6853
[123]	valid_0's l1: 22.6842
[124]	valid_0's l1: 22.6977
[125]	valid_0's l1: 22.7061
[126]	valid_0's l1: 22.7153
[127]	valid_0's l1: 22.714
[128]	valid_0's l1: 22.7192
[129]	valid_0's l1: 22.7183
[130]	valid_0's l1: 22.7227
[131]	valid_0's l1: 22.7318
[132]	valid_0's l1: 22.7111
[133]	valid_0's l1: 22.7076
[134]	valid_0's l1: 22.7104
[135]	valid_0's l1: 22.7185
[136]	valid_0's l1: 22.7207
[137]	valid_0's l1: 22.7195
[138]	valid_0's l1: 22.7223
[139]	valid_0's l1: 22.7249
[140]	valid_0's l1: 22.7165
[141]	valid_0's l1: 22.7303
[142]	valid_0's l1: 22.7397
[143]	valid_0's l1: 22

In [55]:
model.params
model.best_iteration
model.best_score

{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 3.92994872964793e-05,
 'lambda_l2': 0.0008522013002932,
 'num_leaves': 68,
 'feature_fraction': 0.88,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 5,
 'num_iterations': 1000,
 'early_stopping_round': 100}

105

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('l1', 22.512628043393622)])})

In [17]:
y_pred = model.predict(x_test, num_iteration=model.best_iteration)

In [18]:
y_pred

array([349.98515996, 274.54358595, 475.47189709, ..., 558.89891787,
       171.24837743, 213.92791015])

In [19]:
# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame({'R2': r2_score(true, pred),
                          'MAE': mean_absolute_error(true, pred),
                          'MSE': mean_squared_error(true, pred),
                          'RMSE': np.sqrt(mean_squared_error(true, pred))},
                           index = ['scores'])
    return scores

scores = calculate_scores(y_test, y_pred)
print(scores)

              R2        MAE          MSE       RMSE
scores  0.961197  22.512628  1139.313376  33.753716


In [34]:
submitt = model.predict(test_pre, num_iteration=model.best_iteration)

In [45]:
submitt_data = pd.DataFrame(submitt)
submitt_data.head()

,0
0,533.761813
1,300.148075
2,205.382031
3,175.737408
4,302.779893


In [46]:
submitt_data.shape

(9000, 1)

(9000, 1)

In [47]:
id_data = pd.DataFrame(test['id'])
submitt_data = pd.concat([id_data,submitt_data], axis=1)
submitt_data

,id,0
0,0,533.761813
1,1,300.148075
2,2,205.382031
3,3,175.737408
4,4,302.779893
...,...,...
8995,8995,310.327189
8996,8996,303.757823
8997,8997,510.573938
8998,8998,143.923275


In [54]:
submitt_data.columns = ['id','y']
submitt_data.to_csv('../output/submit_1.csv', index=False)